## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-10-23-04-39 +0000,nyt,"Russia Further Restricts Telegram, Escalating ...",https://www.nytimes.com/2026/02/10/world/europ...
1,2026-02-10-23-00-24 +0000,nypost,Australian man ‘lucky to be alive’ after beate...,https://nypost.com/2026/02/10/world-news/austr...
2,2026-02-10-23-00-07 +0000,nyt,F.D.A. Refuses to Review Moderna Flu Vaccine,https://www.nytimes.com/2026/02/10/health/fda-...
3,2026-02-10-22-59-10 +0000,nyt,Georgia Ballot Inquiry Originated With Electio...,https://www.nytimes.com/2026/02/10/us/politics...
4,2026-02-10-22-54-10 +0000,bbc,Ex-Labour comms chief suspended over links to ...,https://www.bbc.com/news/articles/cqxdry8px2xo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2448/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,41
112,epstein,22
328,new,21
65,canada,14
359,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
255,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...,112
284,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...,109
328,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...,105
318,2026-02-10-02-41-35 +0000,nyt,Trump Threatens to Block Opening of New Bridge...,https://www.nytimes.com/2026/02/09/us/politics...,101
93,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,95


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
255,112,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...
133,71,2026-02-10-19-31-53 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
284,68,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...
10,50,2026-02-10-22-46-00 +0000,wsj,"Unsealed document shows role Kurt Olsen, hired...",https://www.wsj.com/politics/elections/stop-th...
202,45,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
334,39,2026-02-10-01-25-52 +0000,latimes,Federal judge strikes down California mask ban...,https://www.latimes.com/california/story/2026-...
9,39,2026-02-10-22-47-54 +0000,nypost,Hochul warns NY won’t chip in more funding to ...,https://nypost.com/2026/02/10/us-news/hochul-w...
292,34,2026-02-10-08-46-21 +0000,wsj,Stock Market Today: Financial Stocks Face AI P...,https://www.wsj.com/livecoverage/stock-market-...
293,32,2026-02-10-08-18-20 +0000,nypost,Fiend sobs over missing Grand Theft Auto video...,https://nypost.com/2026/02/10/world-news/man-s...
336,32,2026-02-10-01-13-00 +0000,wsj,The White House is at loggerheads with Congres...,https://www.wsj.com/politics/policy/proposed-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
